# COMP9318 Project
## Exploratory Data Analysis

In [33]:
import helper as h
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas2arff

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

Read in word data:

In [3]:
data_loc = 'asset/training_data.txt'
words = h.get_words(data_loc)

In [3]:
words.head()

word                pronunciation  \
0          COED                  K OW1 EH2 D   
1       PURVIEW                P ER1 V Y UW2   
2         HEHIR              HH EH1 HH IH0 R   
3      MUSCLING         M AH1 S AH0 L IH0 NG   
4  NONPOISONOUS  N AA0 N P OY1 Z AH0 N AH0 S   

                                  pn_list                  destressed_pn_list  \
0                        [K, OW1, EH2, D]                      [K, OW, EH, D]   
1                     [P, ER1, V, Y, UW2]                   [P, ER, V, Y, UW]   
2                   [HH, EH1, HH, IH0, R]                 [HH, EH, HH, IH, R]   
3            [M, AH1, S, AH0, L, IH0, NG]           [M, AH, S, AH, L, IH, NG]   
4  [N, AA0, N, P, OY1, Z, AH0, N, AH0, S]  [N, AA, N, P, OY, Z, AH, N, AH, S]   

               primary_stress_map            secondary_stress_map  \
0                    [0, 1, 0, 0]                    [0, 0, 1, 0]   
1                 [0, 1, 0, 0, 0]                 [0, 0, 0, 0, 1]   
2                 [0, 1, 0, 0, 0]                 [0, 0, 0, 0, 0]   
3           [0, 1, 0, 0, 0, 0, 0]           [0, 0, 0, 0, 0, 0, 0]   
4  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]   

                        vowel_map                   consonant_map  \
0                    [0, 1, 1, 0]                    [1, 0, 0, 1]   
1                 [0, 1, 0, 0, 1]                 [1, 0, 1, 1, 0]   
2                 [0, 1, 0, 1, 0]                 [1, 0, 1, 0, 1]   
3           [0, 1, 0, 1, 0, 1, 0]           [1, 0, 1, 0, 1, 0, 1]   
4  [0, 1, 0, 0, 1, 0, 1, 0, 1, 0]  [1, 0, 1, 1, 0, 1, 0, 1, 0, 1]   

                                          vector_map  vowel_count ...  R  S  \
0  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...            2 ...  0  0   
1  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, ...            2 ...  0  0   
2  [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ...            2 ...  1  0   
3  [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...            3 ...  0  1   
4  [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...            4 ...  0  1   

   SH  T  TH  V  W  Y  Z  ZH  
0   0  0   0  0  0  0  0   0  
1   0  0   0  1  0  1  0   0  
2   0  0   0  0  0  0  0   0  
3   0  0   0  0  0  0  0   0  
4   0  0   0  0  0  0  1   0  

[5 rows x 57 columns]

In [4]:
words.columns

Index(['word', 'pronunciation', 'pn_list', 'destressed_pn_list',
       'primary_stress_map', 'secondary_stress_map', 'vowel_map',
       'consonant_map', 'vector_map', 'vowel_count', 'consonant_count',
       'type_tag', '1st_letter_idx', 'phoneme_length', 'prefix', 'suffix',
       'primary_stress_idx', 'stressed_vowel', 'AA', 'AE', 'AH', 'AO', 'AW',
       'AY', 'EH', 'ER', 'EY', 'IH', 'IY', 'OW', 'OY', 'UH', 'UW', 'P', 'B',
       'CH', 'D', 'DH', 'F', 'G', 'HH', 'JH', 'K', 'L', 'M', 'N', 'NG', 'R',
       'S', 'SH', 'T', 'TH', 'V', 'W', 'Y', 'Z', 'ZH'],
      dtype='object')

### Facts

Most common stressed vowel in descending order. Compare against total count of particular vowel, can then take proability that vowel will be stressed

In [4]:
# Get count where particular vowel is the stress vowel and count where vowel is in existance
stressed_vowel_count = words.groupby('stressed_vowel')['word'].count().sort_values(ascending=False)
total_vowel_count = words[list(h.vowels)].sum()

# Join the two frames
vowel_counts = pd.concat((stressed_vowel_count,total_vowel_count), axis=1, join='inner')
vowel_counts.columns = ['stressed_vowel_count','total_vowel_count']
vowel_counts['vowel_count_prob'] = vowel_counts.stressed_vowel_count/vowel_counts.total_vowel_count
vowel_counts.sort_values(['vowel_count_prob'],ascending=False)

stressed_vowel_count  total_vowel_count  vowel_count_prob
AE                  6442               8104          0.794916
EH                  7892              10090          0.782161
AA                  6331               8957          0.706821
AO                  2952               4260          0.692958
EY                  3167               4728          0.669839
OY                   297                446          0.665919
UW                  2328               3522          0.660988
AW                   853               1308          0.652141
UH                   608                933          0.651661
AY                  2502               4178          0.598851
OW                  3185               7092          0.449098
IH                  5805              16630          0.349068
IY                  3515              12281          0.286214
ER                  1645              11506          0.142969
AH                  2478              23623          0.104898

Find patterns in training data, build list of all possible consecutive sequences

In [28]:
def sub_string(pronunciation_list,length):
    return tuple(zip(*(pronunciation_list[i:] for i in range(length))))

def get_sequences(phoneme_series):
    sequences = {}
    max_length = max(phoneme_series.str.len())
    for i in range(2, max_length + 1):
        for pn_list in phoneme_series:
            # Next iteration if pn_list is shorter then the sequence length be built
            if len(pn_list) < i:
                continue
            word_sequences = sub_string(pn_list,i)
            for seq in word_sequences:
                sequences[seq] = sequences.get(seq,0) + 1
    return sequences

def in_list(pn_list,sequence):
    if pn_list in sequence:
        return 1
    return 0

def is_primary(sequence):
    for phoneme in sequence:
        if '1' in phoneme:
            return True
    return False

# Generate Dataframe with all destressed sequence possibilities and get counts
destressed_sequences = get_sequences(words.destressed_pn_list)
destressed_sequence_df = pd.DataFrame(list(destressed_sequences.items()),columns=['Destressed_Sequence','Destressed_Sequence_Count'])
destressed_sequence_df = destressed_sequence_df.set_index('Destressed_Sequence')
# Generate Dataframe with all sequence possibiities and get counts, flag if primary stress in sequence
sequences = get_sequences(words.pn_list)
sequence_df = pd.DataFrame(list(sequences.items()),columns=['Sequence','Sequence_Count'])

# Return True is sequence has primary stress in it
sequence_df['Is_Primary'] = sequence_df.Sequence.apply(is_primary)
sequence_df['Destressed_Sequence'] = sequence_df.Sequence.apply(h.filter_stress)
sequence_df.Destressed_Sequence = sequence_df.Destressed_Sequence.apply(h.as_tuple)
sequence_df = sequence_df.query('Is_Primary == True').set_index('Destressed_Sequence')

# Join 
sequences_df = sequence_df.join(destressed_sequence_df)

# Get probability that sequence if exists will be stressed
sequences_df['Sequence_Stress_Probability'] = sequences_df.Sequence_Count/sequences_df.Destressed_Sequence_Count
sequences_df['Sequence_Length'] = sequences_df.Sequence.str.len()

In [55]:
valid_seqs = sequences_df.query('Sequence_Stress_Probability  < 1 & Destressed_Sequence_Count > 5')
valid_seqs.sort_values(by=['Sequence_Stress_Probability','Destressed_Sequence_Count'],ascending=False)
#sequences_df.query('Sequence_Stress_Probability > 0.5  & Destressed_Sequence_Count > 5').sort_values(by='Destressed_Sequence',ascending=False)
sequences_df.query('Sequence_Stress_Probability > 0.5  & Sequence_Length < 3')

# Doorknob
sequences_df.loc[[('UW','R'),('AA','B')]]

# Rabon
#RABON:R AA0 B AO1 N
sequences_df.loc[[('AA','B'),('AO','N')]]

#ABILA:AA0 B IY1 L AH0
sequences_df.loc[[('AA','B'),('IY','L')]]




Sequence  Sequence_Count Is_Primary  \
Destressed_Sequence                                        
(AA, B)              (AA1, B)             182       True   
(IY, L)              (IY1, L)             276       True   

                     Destressed_Sequence_Count  Sequence_Stress_Probability  \
Destressed_Sequence                                                           
(AA, B)                                    239                     0.761506   
(IY, L)                                    506                     0.545455   

                     Sequence_Length  
Destressed_Sequence                   
(AA, B)                            2  
(IY, L)                            2


Only show sequences that appear in atleast 1% of words in training data. Compare sequence with stresses included.

In [25]:
sequences_df.query('Is_Primary == True').sort_values(by='Sequence_Count',ascending=False)

Sequence  \
Destressed_Sequence                                                         
(AA, R)                                                          (AA1, R)   
(AO, R)                                                          (AO1, R)   
(AE, N)                                                          (AE1, N)   
(EH, N)                                                          (EH1, N)   
(EH, L)                                                          (EH1, L)   
(EH, R)                                                          (EH1, R)   
(AA, N)                                                          (AA1, N)   
(IH, N)                                                          (IH1, N)   
(R, EH)                                                          (R, EH1)   
(K, AA)                                                          (K, AA1)   
(IH, L)                                                          (IH1, L)   
(R, AE)                                                          (R, AE1)   
(IY, N)                                                          (IY1, N)   
(R, IH)                                                          (R, IH1)   
(AE, L)                                                          (AE1, L)   
(OW, L)                                                          (OW1, L)   
(L, AE)                                                          (L, AE1)   
(AA, L)                                                          (AA1, L)   
(EH, K)                                                          (EH1, K)   
(EH, S)                                                          (EH1, S)   
(EH, T)                                                          (EH1, T)   
(L, EH)                                                          (L, EH1)   
(K, AE)                                                          (K, AE1)   
(R, IY)                                                          (R, IY1)   
(AE, K)                                                          (AE1, K)   
(L, IH)                                                          (L, IH1)   
(R, AA)                                                          (R, AA1)   
(M, AE)                                                          (M, AE1)   
(R, EY)                                                          (R, EY1)   
(S, EH)                                                          (S, EH1)   
...                                                                   ...   
(G, OW, L, D, S, T, AH)                         (G, OW1, L, D, S, T, AH0)   
(G, OW, L, D, S, T, AH, N)                   (G, OW1, L, D, S, T, AH0, N)   
(G, OW, L, D, S, T, AY)                         (G, OW1, L, D, S, T, AY2)   
(G, OW, L, D, S, T, AY, N)                   (G, OW1, L, D, S, T, AY2, N)   
(G, OW, L, D, HH, AE, M, ER)              (G, OW1, L, D, HH, AE2, M, ER0)   
(G, OW, L, D, HH, AE, M)                       (G, OW1, L, D, HH, AE2, M)   
(G, OW, L, D, HH, AE)                             (G, OW1, L, D, HH, AE2)   
(G, OW, L, D, HH)                                      (G, OW1, L, D, HH)   
(G, OW, L, D, AH, N, TH, AA)              (G, OW1, L, D, AH0, N, TH, AA2)   
(G, OW, L, D, AH, N, TH, AA, L)        (G, OW1, L, D, AH0, N, TH, AA2, L)   
(G, OW, L, D, B, AW)                               (G, OW1, L, D, B, AW2)   
(G, OW, L, D, B, AW, M)                         (G, OW1, L, D, B, AW2, M)   
(G, OW, L, D, B, ER, G, ER)                (G, OW1, L, D, B, ER0, G, ER0)   
(G, OW, L, D, B, L)                                  (G, OW1, L, D, B, L)   
(G, OW, L, D, B, L, AH)                         (G, OW1, L, D, B, L, AH0)   
(G, OW, L, D, B, L, AH, T)                   (G, OW1, L, D, B, L, AH0, T)   
(G, OW, L, D, ER)                                     (G, OW1, L, D, ER0)   
(G, OW, L, D, F, AA)                               (G, OW1, L, D, F, AA2)   
(G, OW, L, D, F, AA, R)                         (G, OW1, L, D, F, AA2, R)   
(G, OW, L, D, F, AA, R, B)                   (G, OW1, L, D, F

### Plots

In [20]:
%matplotlib inline


def scatter_plot(x,y):
    fig = plt.figure()
    ax  = fig.add_subplot(1,1,1)
    ax.scatter(x,y)
    plt.title("{} vs {}".format(y.name,x.name))
    plt.xlabel(x.name)
    plt.ylabel(y.name)
    plt.show()
    return

def histogram(data):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.hist(data,bins='auto')
    plt.title('{} Distribution'.format(data.name))
    plt.xlabel('{}'.format(data.name))
    #plt.xlim(0,bins_count)
    plt.show()
    return    

In [ ]:
for tag in words.type_tag.unique():
    tag_df = words[words['type_tag'] == tag]
    print(tag)
    scatter_plot(tag_df['1st_letter_idx'], tag_df.primary_stress_idx)

In [ ]:
histogram(words.primary_stress_idx.apply(int))

In [ ]:
histogram(sequence_df[sequence_df.Sequence_Count > 0].Sequence_Count)

KeyboardInterrupt: 